In [ ]:
import os
import math
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
import datetime as dt
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import json
import time
import matplotlib.pyplot as plt
from keras.models import  load_model
from  sklearn import  metrics
import itertools 

class Timer():   

	def __init__(self):
		self.start_dt = None

	def start(self):       
		self.start_dt = dt.datetime.now()

	def stop(self):
		end_dt = dt.datetime.now()
		print('Time taken: %s' % (end_dt - self.start_dt))

In [ ]:
class Model():
	"""LSTM 模型"""

	def __init__(self):
		self.model = Sequential()

	def load_model(self, filepath):
		print('[Model] Loading model from file %s' % filepath)
		self.model = load_model(filepath)

	def build_model(self, configs):
		timer = Timer()
		timer.start()

		for layer in configs['model']['layers']:
			neurons = layer['neurons'] if 'neurons' in layer else None
			dropout_rate = layer['rate'] if 'rate' in layer else None
			activation = layer['activation'] if 'activation' in layer else None
			return_seq = layer['return_seq'] if 'return_seq' in layer else None
			input_timesteps = layer['input_timesteps'] if 'input_timesteps' in layer else None
			input_dim = layer['input_dim'] if 'input_dim' in layer else None

			if layer['type'] == 'dense':
				self.model.add(Dense(neurons, activation=activation))
			if layer['type'] == 'lstm':
				self.model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences=return_seq))
			if layer['type'] == 'dropout':
				self.model.add(Dropout(dropout_rate))

		self.model.compile(loss=configs['model']['loss'], optimizer=configs['model']['optimizer'])

		print('[Model] Model Compiled')
		timer.stop()
		
		return self.model
		

	def train(self, x, y,x_val,y_val, epochs, batch_size, save_dir):
		timer = Timer()
		timer.start()
		print('[Model] %s epochs, %s batch size' % (epochs, batch_size))
		
		save_fname = os.path.join(save_dir, '%s-e%s.h5' % (dt.datetime.now().strftime('%d%m%Y-%H%M%S'), str(epochs)))
		callbacks = [
			EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=4),
			ModelCheckpoint(filepath=save_fname, monitor='val_loss', save_best_only=True)
		]

		self.model.fit(
			x,
			y,
			validation_data =(x_val,y_val),
			epochs=epochs,
			batch_size=batch_size,
			callbacks=callbacks
		)

		timer.stop()

	def predict_point_by_point(self, data,debug=False):
		if debug == False:
			print('[Model] Predicting Point-by-Point...')
			predicted = self.model.predict(data)
			predicted = np.reshape(predicted, (predicted.size,))
		else:
			print('[Model] Predicting Point-by-Point...')
			print (np.array(data).shape)
			predicted = self.model.predict(data)
			print (np.array(predicted).shape)
			predicted = np.reshape(predicted, (predicted.size,))
			print (np.array(predicted).shape)
		return predicted

In [ ]:
class DataLoader():

    def __init__(self, filename, split, cols):
        dataframe = pd.read_csv(filename)
        i_split = int(len(dataframe) * split)

        self.data_train = dataframe.get(cols).values[:i_split]
        self.data_test  = dataframe.get(cols).values[i_split:]
        self.all_data = dataframe.get(cols)
        self.len_all  = len(self.all_data)
        self.len_train  = len(self.data_train)
        self.len_test   = len(self.data_test)
        self.len_train_windows = None
    
    def get_all_data(self,seq_len,normalise):

        data_windows = []
        for i in range(self.len_all - seq_len):
            data_windows.append(self.all_data[i:i+seq_len])

        data_windows = np.array(data_windows).astype(float)

        data_windows = self.normalise_windows(data_windows, single_window=False) if normalise else data_windows

        x = data_windows[:, :-1,1:]
        y = data_windows[:, -1, [0]]
        return x,y        


    def get_test_data(self, seq_len, normalise,predict_len):

        data_x_raise = []
        data_y_raise = []
        data_x_down = []
        data_y_down = []

        for i in range(self.len_test - seq_len):   
            x, y = self._next_window_test(i, seq_len, normalise)
            x0,y0 = self.all_next_window_test(i, seq_len, normalise)
            if y0[-(1+predict_len)] >= y0[-(2+predict_len)]:
                data_x_raise.append(x)
                data_y_raise.append(y)
            if y0[-(1+predict_len)] < y0[-(2+predict_len)]:
                data_x_down.append(x)
                data_y_down.append(y)                
        return np.array(data_x_raise), np.array(data_y_raise),np.array(data_x_down), np.array(data_y_down)
    
    def all_next_window_test(self, i, seq_len, normalise):
        window = self.data_test[i:i+seq_len]
        window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = window[:,1:]
        y = window[:, [0]]
        return x, y    

    def _next_window_test(self, i, seq_len, normalise):
        window = self.data_test[i:i+seq_len]
        window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = window[:-1,1:]
        y = window[-1, [0]]
        return x, y

    def get_train_data(self, seq_len, normalise,predict_len):

        data_x_raise = []
        data_y_raise = []
        data_x_down = []
        data_y_down = []

        for i in range(self.len_train - seq_len):   
            x, y = self._next_window(i, seq_len, normalise)
            x0,y0 = self.all_next_window(i, seq_len, normalise)
            if y0[-(1+predict_len)] >= y0[-(2+predict_len)]:
                data_x_raise.append(x)
                data_y_raise.append(y)
            if y0[-(1+predict_len)] < y0[-(2+predict_len)]:
                data_x_down.append(x)
                data_y_down.append(y)                
        return np.array(data_x_raise), np.array(data_y_raise),np.array(data_x_down), np.array(data_y_down)


    
    def all_next_window(self, i, seq_len, normalise):
        window = self.data_train[i:i+seq_len]
        window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = window[:,1:]
        y = window[:, [0]]
        return x, y    

    def _next_window(self, i, seq_len, normalise):
        window = self.data_train[i:i+seq_len]
        window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = window[:-1,1:]
        y = window[-1, [0]]
        return x, y

    def normalise_windows(self, window_data, single_window=False):
        normalised_data = []
        window_data = [window_data] if single_window else window_data

        for window in window_data:
            normalised_window = []
            for col_i in range(window.shape[1]):
                normalised_col = [((float(p) / float(window[0, col_i])) - 1) for p in window[:, col_i]]   
                normalised_window.append(normalised_col)
            normalised_window = np.array(normalised_window).T 
            normalised_data.append(normalised_window)
        return np.array(normalised_data)

In [ ]:
def plot_results(predicted_data, true_data):     
    fig = plt.figure(facecolor='white')
    fig = plt.figure(figsize=(16, 5))
    ax = fig.add_subplot(111)

    ax.plot(true_data, label='True Data')
    plt.plot(predicted_data, label='Prediction')

    ax.set_xlabel('Time Series') 
    ax.set_ylabel('Water Level')  
    plt.legend()
    plt.savefig('results_2.png')

def GetRMSE(y_hat,y_test):
    sum = np.sqrt(metrics.mean_squared_error(y_test, y_hat))
    return  sum

def GetMAE(y_hat,y_test):
    sum = metrics.mean_absolute_error(y_test, y_hat)
    return  sum

def GetMAPE(y_hat,y_test):
    sum = np.mean(np.abs((y_hat - y_test) / y_test)) * 100
    return sum

def GetMAPE_Order(y_hat,y_test):
    zero_index = np.where(y_test == 0)
    y_hat = np.delete(y_hat,zero_index[0])
    y_test = np.delete(y_test,zero_index[0])
    sum = np.mean(np.abs((y_hat - y_test) / y_test)) * 100
    return sum

def nash_sutcliffe(obs, sim):
    obs_mean = np.mean(obs)
    numerator = np.sum((obs - sim) ** 2)
    denominator = np.sum((obs - obs_mean) ** 2)
    nse = 1 - numerator / denominator
    return nse

def calculate_kge(observations, simulations):
    rho = np.corrcoef(observations, simulations)[0, 1]
    beta = np.std(simulations) / np.std(observations)
    alpha = np.mean(simulations) / np.mean(observations)
    kge = 1 - np.sqrt((rho - 1)**2 + (beta - 1)**2 + (alpha - 1)**2)
    return kge

In [ ]:
def get_normalize_parameters(train_year, test_year,all_year,filename = "C:/Users/ljm/Desktop/LSTM模型的各种尝试/data/数据处理/2013-2021,1days训练测试.xlsx" ):     
    data_train = pd.read_excel(filename)
    df = data_train.iloc[int((all_year-train_year-test_year) * data_train.shape[0]/all_year) : , 1:]
    miu = []
    sigma = []
    split_propotion = train_year/(train_year+test_year)
    for i in range(df.shape[1]):
        mean = np.mean(df.iloc[:int(split_propotion*df.shape[0]),i])
        variance = np.var(df.iloc[:int(split_propotion*df.shape[0]),i])
        miu.append(mean)
        sigma.append(variance)
    return miu[0],sigma[0]**0.5,split_propotion

def F_normalise(prediction,y_test,sigma,miu):
    true_predictions_pointbypoint = []
    true_y_test = []
    for i in range(len(prediction)):
        true_predictions_pointbypoint.append(sigma * prediction[i] + miu)        
    for j in range(len(prediction)):
        true_y_test.append(sigma * y_test[j] + miu)
    return true_predictions_pointbypoint,true_y_test

## Train

In [ ]:
miu,sigma,split_propotion = get_normalize_parameters(train_year = 5,test_year = 3,all_year = 8,filename = "C:/Users/ljm/Desktop/LSTM模型的各种尝试/data/数据处理/5-3-2-2d-selfLuoshan-train.xlsx")
print(miu,sigma,split_propotion)
predict_lenth = 3
s_lenghth = [10,30,60]
b_size = [4,8,32,128]
net1 = [8,32,128]
net2 = [8,32,128]
net3 = [8,32,128]
d_out = [0.2,0.3]
grid_params = list(itertools.product(s_lenghth, b_size, net1,net2,net3,d_out))

best_MAE = 1000
RMSE_with = 1000
MAE_list = []
for params in grid_params:
	configs = {
    	"data": {
			"filename": "Robust0.4-5-3-2-Hankou-train.csv",
			"columns": [
				"螺山日均水位",
				'自相关螺山水位',
				"枝城日均流量",
				"津市（二）日均流量",
				"湘潭日均流量",
				"桃源日均流量",
				"桃江（二）日均流量"],
			"sequence_length": params[0],"train_test_split": split_propotion,"normalise": False},
		"training": {"epochs": 1000,"batch_size": params[1]},
		"model": {"loss": "mse","optimizer": "adam","save_dir": "5-3-2-2-0.4-Hankou-divided",
			"layers": [
				{"type": "lstm","neurons": params[2],"input_timesteps": params[0]-1,"input_dim": 6,"return_seq": True},
				{"type": "dropout","rate": params[5]},
				{"type": "lstm","neurons": params[3],"return_seq": True},
				{"type": "lstm","neurons": params[4],"return_seq": False},
				{"type": "dropout","rate": params[5]},
				{"type": "dense","neurons": 1,"activation": "linear"}]}}
	
	data = DataLoader(
		os.path.join('data', configs['data']['filename']),
		configs['data']['train_test_split'],
		configs['data']['columns'])
	model = Model()
	mymodel = model.build_model(configs)
	x, y,x2,y2 = data.get_train_data(seq_len=configs['data']['sequence_length'],normalise=configs['data']['normalise'],predict_len = predict_lenth)
	x_test, y_test,x2_test,y2_test = data.get_test_data(seq_len=configs['data']['sequence_length'],normalise=configs['data']['normalise'],predict_len = predict_lenth)

	model.train(x,y,x_test,y_test,
	epochs = configs['training']['epochs'],
	batch_size = configs['training']['batch_size'],
	save_dir = configs['model']['save_dir'])
		
	predictions_pointbypoint = model.predict_point_by_point(x_test,debug=False)
	true_predictions_pointbypoint,true_y_test = F_normalise(prediction = predictions_pointbypoint,y_test=y_test,sigma=sigma,miu=miu)

	model = Model()
	mymodel = model.build_model(configs)
	model.train(x2,y2,x2_test,y2_test,
	epochs = configs['training']['epochs'],
	batch_size = configs['training']['batch_size'],
	save_dir = configs['model']['save_dir'])

	predictions_pointbypoint = model.predict_point_by_point(x2_test,debug=False)
	true_predictions_pointbypoint2,true_y_test2 = F_normalise(prediction = predictions_pointbypoint,y_test=y2_test,sigma=sigma,miu=miu)
	pre = true_predictions_pointbypoint + true_predictions_pointbypoint2
	true = true_y_test + true_y_test2
	MAE_index = GetMAE(pre , true)
	MAE_list.append(MAE_index)
	if MAE_index < best_MAE:
		best_MAE = MAE_index
		RMSE_with = GetRMSE(true_predictions_pointbypoint, true_y_test)
		best_params = params
print(best_MAE)
print(RMSE_with)
print(best_params)
df_MAE = pd.DataFrame(MAE_list)
df_MAE.to_excel('MAE_5-3-2-Hankou-divided-0.4.xlsx',index=False)

## Test and Plot

In [ ]:
predict_lenth = 3
root_dir = "C:/Users/ljm/Desktop/LSTM模型的各种尝试/selected_model/新的10年实验/5-3-2-2-OnlySelf-divided"

#Different sequence length for different model dicide different figure 
sequence_length_list = [10, 10]
MAE = []
RMSE = []
NSE = []
num = 0
sum_prediction = []
for root, dirs, files in os.walk(root_dir):
    for file in files:
        file_name = file
        model_one = load_model(os.path.join(root_dir, file_name))
        data = DataLoader(os.path.join('data', configs['data']['filename']),configs['data']['train_test_split'],
        configs['data']['columns'])

        x_test, y_test,x2_test,y2_test,order_list,lenth = data.get_all_data(seq_len=sequence_length_list[num],normalise=configs['data']['normalise'],predict_len = predict_lenth)

        if num%2 == 0:
            predictions_pointbypoint = model_one.predict(x_test)
            true_predictions_pointbypoint,true_y_test = F_normalise(prediction = predictions_pointbypoint,y_test=y_test,sigma=sigma,miu=miu)
            prediction = true_predictions_pointbypoint
            reality = true_y_test
        if num%2 == 1:
            predictions_pointbypoint = model_one.predict(x2_test)
            true_predictions_pointbypoint,true_y_test = F_normalise(prediction = predictions_pointbypoint,y_test=y2_test,sigma=sigma,miu=miu)
            prediction += true_predictions_pointbypoint
            reality += true_y_test
        
            MAE.append(GetMAE(prediction, reality))
            RMSE.append(GetRMSE(prediction, reality))
            NSE.append(nash_sutcliffe(reality,prediction))

            high_level_true = []
            high_level_pre = []
            ordinary_level_true = []
            ordinary_level_pre = []
            low_level_true = []
            low_level_pre = []
            for i  in range (len(reality)):
                if reality[i] >= (miu + sigma):
                    high_level_true.append(reality[i])
                    high_level_pre.append(prediction[i])
                
                if reality[i] <= (miu - sigma):
                    low_level_true.append(reality[i])
                    low_level_pre.append(prediction[i])
                
                else:
                    ordinary_level_true.append(reality[i])
                    ordinary_level_pre.append(prediction[i])

            print('High',GetMAE(high_level_pre, high_level_true), GetRMSE(high_level_pre, high_level_true))
            print('Ordinary',GetMAE(ordinary_level_pre, ordinary_level_true), GetRMSE(ordinary_level_pre, ordinary_level_true))
            print('Low',GetMAE(low_level_pre, low_level_true), GetRMSE(low_level_pre, low_level_true))


            #Ensemble Model
            order_list_predict = [0] * lenth
            order_list_true = [0] *  lenth
            for i in range (len(order_list)): 
                order = order_list[i]
                order_list_predict[order] = prediction[i]
                order_list_true[order] = reality[i]

            largest_sequence = max(sequence_length_list)
            order_list_predict = order_list_predict[(largest_sequence-sequence_length_list[num]):]

            if num == 1:
                sum_prediction = order_list_predict
            else :
                list = []
                for i in range(len(sum_prediction)):
                    list.append((sum_prediction[i]+order_list_predict[i]))
                sum_prediction = list
            order_list_true = order_list_true[(largest_sequence-sequence_length_list[num]):]


        num+=1
    
average_prediction = []
for i in range(len(sum_prediction)):
    average_prediction.append((sum_prediction[i]/(num/2)))
plot_results(average_prediction,order_list_true)
print("全部数据MAE(绝对误差的平均值）为", GetMAE(average_prediction, order_list_true))
print("全部数据RMSE（均方根误差）为", GetRMSE(average_prediction, order_list_true))
print("NSE为",nash_sutcliffe(order_list_true,average_prediction))

MAE_df = pd.DataFrame(MAE)
RMSE_df = pd.DataFrame(RMSE)
NSE_df = pd.DataFrame(NSE)
MAE_df = MAE_df.rename(columns={0:'MAE'})
RMSE_df = RMSE_df.rename(columns={0:'RMSE'})
result = MAE_df.join(RMSE_df).join(NSE_df) 
result.to_excel('Result.xlsx',index  = False)                     
print(MAE)
print(RMSE)

### Plot Figure6 in the paper

In [ ]:
#查看涨落水各自的精度
predict_lenth = 3

root_dir = 'C:/Users/ljm/Desktop/LSTM模型的各种尝试/selected_model/新的10年实验/5-3-2-2-self-divided/best'

#不同模型需要改这个 
sequence_length_list = [10, 10]

file_name = '1-1.h5'
model_one = load_model(os.path.join(root_dir, file_name))
data = DataLoader(os.path.join('data', configs['data']['filename']),configs['data']['train_test_split'],
configs['data']['columns'])

#预测集用all,训练集用test
x_test, y_test,x2_test,y2_test,order_list,lenth = data.get_all_data(seq_len=sequence_length_list[0],normalise=configs['data']['normalise'],predict_len = predict_lenth)

predictions_pointbypoint = model_one.predict(x2_test)
true_predictions_pointbypoint,true_y_test = F_normalise(prediction = predictions_pointbypoint,y_test=y2_test,sigma=sigma,miu=miu)

file_name = '1-2.h5'
model_one = load_model(os.path.join(root_dir, file_name))
data = DataLoader(os.path.join('data', configs['data']['filename']),configs['data']['train_test_split'],
configs['data']['columns'])
predictions_pointbypoint = model_one.predict(x2_test)
true_predictions_pointbypoint2,true_y_test2 = F_normalise(prediction = predictions_pointbypoint,y_test=y2_test,sigma=sigma,miu=miu)

print(GetMAE(true_predictions_pointbypoint, true_y_test))
print(GetMAE(true_predictions_pointbypoint2, true_y_test2))


X = [i for i in range(len(true_predictions_pointbypoint))]
import matplotlib.lines as mlines
#散点图
fig = plt.figure(figsize=(11, 8))
plt.scatter(X, true_predictions_pointbypoint, s=5, c='orange',label='Rising' ,marker='^', vmax=None, alpha=None, linewidths=None,  edgecolors=None, plotnonfinite=False, data=None)
plt.scatter(X, true_predictions_pointbypoint2, s=5, c='b', label='Recession' ,marker='s', vmax=None, alpha=None, linewidths=None,  edgecolors=None, plotnonfinite=False, data=None)
plt.ylabel('Water Level(m)', fontsize=16, fontweight='bold')  # y 轴标签


legend_handle1 = mlines.Line2D([], [], color='orange', marker='^', linestyle='None',
                              markersize=7, label='Rising')  # 调整markersize以改变图例中的标记大小
legend_handle2 = mlines.Line2D([], [], color='blue', marker='s', linestyle='None',
                              markersize=7, label='Recession')  # 调整markersize以改变图例中的标记大小

plt.legend(handles=[legend_handle1, legend_handle2], loc='upper right', fontsize=14)